In [ ]:
!pip install pydriller==1.15 nltk

     |████████████████████████████████| 62 kB 988 kB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 170 kB 19.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [ ]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
import pandas as pd
import json
import os
from pydriller import RepositoryMining
from nltk import word_tokenize
from datetime import datetime

In [ ]:
#globals
DATA_PATH = './drive/MyDrive/Moataz Chouchen/MCR projects/Code review refactoring and effort/data '

In [ ]:
def compute_statistics(project_name = 'QT',considered_status = ['merged-changes','abondaned-changes'],keywords= {'refactoring' : ['refactor']}): 
    subject_keywords_count = {cat : {word : 0 for word in  keywords[cat]} for cat in keywords}
    description_keywords_count = {cat : {word : 0 for word in  keywords[cat]} for cat in keywords}
    result =[]# pd.DataFrame(columns = ['id','#reviewers','#messages','churn','#revisions','#files',
             #                        'duration','len_description','description_wordcount',
             #                        'subject categories','subject keywords','description categories',
              #                       'description keywords'])
    for current_status in considered_status : 
        print('current status:',current_status)
        data_path = os.path.join(DATA_PATH,'raw data',project_name,'review data',current_status)
        for  filename in os.listdir(data_path):
            if '.json' in filename : 
                print('processing file:',filename)
                raw_data = json.load(open(os.path.join(data_path,filename)))
                for change_data in raw_data : 
                    change_subject = extract_change_subject(change_data)
                    change_description = extract_change_description(change_data,os.path.join(DATA_PATH,'raw data',project_name,'git repos'))
                    subject_kwds_count,subject_found_cats,subject_found_kwds  = keywrods_count(keywords,change_subject)
                    if change_description != None :
                        desc_kwds_count,desc_found_cats,desc_found_kwds  = keywrods_count(keywords,change_description)
                    else : 
                        change_description = ''
                        desc_kwds_count = {cat : {word : 0 for word in  keywords[cat]} for cat in keywords}
                        desc_found_cats = ''
                        desc_found_kwds = ''
                    if len(subject_found_kwds) > 0 or len(desc_found_kwds) > 0:
                        print('bingo')
                        nb_messages, discussion_length = compute_exchanged_messages(change_data)
                        new_row = {
                            'id': change_data['id'],
                            '#reviewers' : compute_reviewers_number(change_data),
                            '#messages' : nb_messages,
                            'len_messages' :discussion_length, 
                            'churn' : compute_code_churn(change_data),
                            '#revisions' : compute_nb_revisions(change_data),
                            '#files' :  compute_modified_files_number(change_data),
                            'duration' : compute_duration(change_data),
                            'len_description': len(change_description),
                            'subject categories' : subject_found_cats,
                            'subject keywords' : subject_found_kwds,
                            'description categories' :desc_found_cats,
                            'description keywords' : desc_found_kwds
                        }
                        result.append(new_row)

    return result,subject_keywords_count, description_keywords_count

def compute_duration(change_data) : 
    update_time = datetime.fromisoformat(change_data['updated'][:-3])
    create_time = datetime.fromisoformat(change_data['created'][:-3])
    duration = update_time - create_time
    return duration.total_seconds()*1.0/3600
    
def compute_code_churn(change_data) : 
    #chrun for the first revision
    first_revision_data_files = extract_revision(change_data,n_revision = 1)['data']['files']
    churn = 0 
    for filename in first_revision_data_files : 
        churn += safe_get(first_revision_data_files[filename],'lines_deleted',0)
        churn += safe_get(first_revision_data_files[filename],'lines_inserted',0)
    
    return churn 

    return change_data['deletions'] + change_data['insertions']
def compute_reviewers_number(change_data) : 
    reviewers = safe_get(safe_get(change_data,'reviewers',{}),'REVIEWER',[])
    nb_reviewers = 0 
    for reviewer in reviewers : 
        if 'email' in reviewer : 
            nb_reviewers += 1 
    return nb_reviewers 

def compute_exchanged_messages(change_data) :
    nb_messages = 0 
    discussion_length = 0 
    for message in change_data['messages'] : 
        if 'email' in message['author'] : 
            nb_messages += 1 
            discussion_length += len(message['message'])
    return nb_messages, discussion_length

def compute_nb_revisions(change_data) : 
    return len(change_data['revisions'])

def compute_modified_files_number(change_data) : 
    return len(extract_revision(change_data,n_revision = 1)['data']['files'])

def keywrods_count(keywords,text) : 
    splitted_text = [token.lower() for token in tokenize_text(text) ]
    kwds_count = {cat : {word : 0 for word in  keywords[cat]} for cat in keywords}
    found_cats = ''
    found_kwds = ''
    for cat in keywords : 
        found_category = False 
        for keyword in keywords[cat] : 
            if keyword.lower() in splitted_text : 
                kwds_count[cat][keyword] += 1 
                if not (cat in found_cats) : 
                     found_category = True 
                     found_cats += cat+'/'
                found_kwds += keyword + ','
        if found_category :
            found_kwds +='|'
    return kwds_count,found_cats,found_kwds 

def update_count(global_count,update) : 
    for cat in global_count : 
        for keyword in global_count[cat] : 
            global_count[cat][keyword] += update[cat][keyword]
    return global_count

def read_keywords(kwd_path) : 
    return json.load(kwd_path)
    
def extract_change_subject(change_data) :
    return change_data["subject"]

def extract_change_description(change_data,repos_path) : 
    try:
        commit_id = change_data['current_revision']
    except Exception as e : 
        print(e)
        print(change_data)
        return None 
    repo_name = extract_repo_name(change_data)

    try: 
        for commit in RepositoryMining(os.path.join(repos_path,repo_name),single=commit_id).traverse_commits():
            return commit.msg
    except Exception as e:
        print(e)
        print('problem with:')
        print('******* change id:',change_data['id'])
        #print('******* repo name:',repo_name)
        #print('******* commit hash:',commit_id)
        
        return None 
        
def extract_revision(change_data,n_revision = 1) :
    for revision_id in  change_data['revisions'] :
        if change_data['revisions'][revision_id]['_number'] == n_revision: 
            return {'commit' : revision_id, 'data' :change_data['revisions'][revision_id] }
    return None 

def extract_repo_name(change_data) : 
    
    id = change_data['id']
    try: 
        if '%2F' in id:    
            repo_name = id.split('~')[0].split('%2F')[1] 
        else: 
            repo_name = id.split('~')[0]
        return repo_name
    except : 
        print(id)
        return None

def tokenize_text(text) : 
    return word_tokenize(text)

def safe_get(data,key,default=None) : 
    try:
      if key in data: 
          return data[key]
      return default
    except : 
        print(data) 

In [ ]:
res,subject_count,desc_count = compute_statistics(considered_status = ['abandoned-changes'])

Streaming output truncated to the last 5000 lines.
******* change id: qt%2Fqt5~5.12.4~Iafdecb0571779077b722bde7521da681f00f6f60
SHA b'7edbbdfb8e63314c8e8cb90de17142f969f63cb4' could not be resolved, git returned: b'7edbbdfb8e63314c8e8cb90de17142f969f63cb4 missing'
problem with:
******* change id: qt-creator%2Fqt-creator~master~I75c4b2b38fec209519adb563d579ecd3a17fe042
SHA b'70e516ba191175df6f54f47ae3215b04a8e61826' could not be resolved, git returned: b'70e516ba191175df6f54f47ae3215b04a8e61826 missing'
problem with:
******* change id: qt%2Fqtlottie~dev~I6931944468757e48fa0426ee8f16c985eeb32d3d
Lock for file '/content/drive/MyDrive/Moataz Chouchen/MCR projects/Code review refactoring and effort/data /raw data/QT/git repos/qtbase/.git/config' did already exist, delete '/content/drive/MyDrive/Moataz Chouchen/MCR projects/Code review refactoring and effort/data /raw data/QT/git repos/qtbase/.git/config.lock' in case the lock is illegal
problem with:
******* change id: qt%2Fqtbase~5.12~Id4a

KeyError: ignored

In [ ]:
res

NameError: ignored

In [ ]:
subject_count

{'refactoring': {'refactor': 0}}